**BfPortf 01 Simple example**

This example shows how to use [bffortf](https://github.com/answering007/bfportf) to find portfolios using [Sharp ratio](https://en.wikipedia.org/wiki/Sharpe_ratio) as a target function and maximize it

In [2]:
# Import section
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.precision', 3)

from bfportf.utils import scale_return_data
from bfportf.optimization import find_portfolios
from bfportf.targets import sharpe_target
from bfportf.report import create_report

In [3]:
# Load data
prices = pd.read_csv("https://raw.githubusercontent.com/answering007/bfportf/master/examples/prices.csv", index_col=0, parse_dates=['Date'])
returns = prices.pct_change()
returns.dropna(axis=0, inplace=True)
returns.head()

,AA,AAL,AAPL,ABBV,ABEV,ABT,ACB,ACWI,ADBE,ADI,...,XLU,XLV,XLY,XME,XOM,XOP,XPO,XRT,XRX,YUM
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-05,-0.058,-5.564e-04,-2.817e-02,-0.019,-0.018,2.230e-04,0.0,-0.020,-0.005,-1.819e-02,...,-1.223e-02,-0.005,-0.019,-0.037,-0.027,-0.064,-0.031,-0.011,-0.022,-0.020
2015-01-06,0.007,-1.559e-02,9.408e-05,-0.005,0.038,-1.136e-02,0.0,-0.010,-0.020,-2.347e-02,...,6.400e-04,-0.003,-0.010,-0.014,-0.005,-0.030,-0.035,-0.011,-0.013,-0.012
2015-01-07,0.026,-5.660e-04,1.402e-02,0.040,0.017,8.108e-03,0.0,0.013,0.008,1.052e-02,...,9.811e-03,0.024,0.016,0.004,0.010,-0.010,0.004,0.026,0.013,0.033
2015-01-08,0.028,1.226e-02,3.842e-02,0.010,0.011,2.055e-02,0.0,0.016,0.025,1.765e-02,...,6.969e-03,0.017,0.015,0.010,0.017,0.034,0.006,0.013,0.028,0.017
2015-01-09,0.013,-3.056e-02,1.073e-03,-0.027,0.015,-1.051e-02,0.0,-0.006,-0.015,-7.305e-04,...,-6.292e-03,-0.009,-0.012,-0.002,-0.001,0.008,-0.013,-0.018,-0.006,-0.002


In [4]:
# Slit data to train and test
df_train, df_test = train_test_split(returns, test_size=0.2, shuffle=False)
print(f'Train shape: {df_train.shape}, Test shape: {df_test.shape}')
print(f'Train start: {str(df_train.index[0])}, end: {str(df_train.index[-1])}')
print(f'Test start: {str(df_test.index[0])}, end: {str(df_test.index[-1])}')

Train shape: (1691, 500), Test shape: (423, 500)
Train start: 2015-01-05 00:00:00, end: 2021-09-21 00:00:00
Test start: 2021-09-22 00:00:00, end: 2023-05-26 00:00:00


In [5]:
# Scale returns
df_train_scaled, train_weights = scale_return_data(df_train)
df_test_scaled, test_weights = scale_return_data(df_test)

In [6]:
# Run brute force search
found_portfolios = find_portfolios(
    returns=df_train_scaled,
    target_function=sharpe_target,
    maximize=True,
    custom_weights=train_weights,
    max_number=3,
    verbose=True
)
found_portfolios.head()

100%|██████████| 500/500 [00:54<00:00,  9.23it/s]


,symbols,target_function_value
0,"[CPRT, NVDA, SHY]",2.352
1,"[AMZN, NVDA, SHY]",2.323
2,"[NVDA, PGR, SHY]",2.312
3,"[DHR, NVDA, SHY]",2.310
4,"[NDAQ, NVDA, SHY]",2.250


In [7]:
# Select best portfolio symbols
selected_portfolio_symbols = found_portfolios['symbols'][0]

# Prettify some report values
formatting = {
    "PnL": "{:.2%}",
    "CAGR": "{:.2%}",
    "Max drawdown": "{:.2%}",
}

# In-sample results
report = create_report(
    returns=df_train_scaled,
    portfolio_symbols=selected_portfolio_symbols,
    return_weights=train_weights,
    benchmark_symbols=['SPY'],
    custom_formatting=formatting)

report

,Portfolio,Benchmark,Diff
PnL,30.06%,139.61%,-109.55%
CAGR,3.99%,13.90%,-9.91%
Max drawdown,-1.98%,-33.72%,31.74%
Profit factor,1.52,1.182,0.338
Recovery factor,13.323,2.91,10.413
Sharpe ratio,2.352,0.821,1.531
Sortino ratio,3.778,1.135,2.643
Skew,0.302,-0.72,1.022
Kurtosis,6.786,17.398,-10.613


In [8]:
# Out-of-sample results
report = create_report(
    returns=df_test_scaled,
    portfolio_symbols=selected_portfolio_symbols,
    return_weights=test_weights,
    benchmark_symbols=['SPY'],
    custom_formatting=formatting)

report

,Portfolio,Benchmark,Diff
PnL,2.00%,-0.84%,2.84%
CAGR,1.18%,-0.50%,1.69%
Max drawdown,-9.40%,-24.50%,15.10%
Profit factor,1.046,1.013,0.033
Recovery factor,0.23,0.116,0.114
Sharpe ratio,0.274,0.081,0.194
Sortino ratio,0.403,0.114,0.289
Skew,0.249,-0.059,0.308
Kurtosis,2.024,0.806,1.217
